In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
from toy_plot_helpers import ToyPlot
plot = ToyPlot()

## Setting up the engine

We always need to set up an MD engine which knows the details of the system we're simulating. The MD engine always takes a template snapshot as an example of what the system looks like. Importantly, the template includes a `topology`, which contains information about how to calculate the interactions.

In [2]:
import openpathsampling.toy_dynamics as toys

pes = (
    toys.toy_pes.OuterWalls([1.0, 1.0], [0.0, 0.0]) + 
    toys.toy_pes.Gaussian(2.0, [1.0, 4.0], [0.0,1.0]) + 
    toys.toy_pes.Gaussian(2.0, [25.0, 1.0], [0.0, -1.0])
)

topology=paths.ToyTopology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)

template = paths.Snapshot(
    coordinates=np.array([[-0.5, 0.0]]), 
    velocities=np.array([[0.0,0.0]]),
    potential_energy = 0.0, # Important to set energies dimensionless otherwise kcal/mol is default
    kinetic_energy = 0.0,
    topology=topology
)

integ = toys.toy_integrators.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'nsteps_per_frame' : 10
}

toy_eng = toys.toy_engine.ToyEngine(
    options=options,
    template=template
)
toy_eng.initialized = True

toy_eng.current_snapshot = template

Register this engine with the PathMovers

In [3]:
paths.PathMover.engine = toy_eng

## Loading trajectories from a previous calculation

Path sampling calculations take trajectories as input, so we need an input trajectory. We get that by running the `toy_bootstrap.ipynb`. Since that's a more advanced example, don't worry about understanding everything in there. Assuming you have an initial trajectory, all you need in order to run TIS is what's included in this notebook.


In [4]:
store_bootstrap = paths.storage.Storage('toy_bootstrap.nc', mode='r')

# note that the next three lines could be combined into one; they're separated here for educational purposes
final_sampleset = store_bootstrap.sampleset[len(store_bootstrap.sampleset)-1]
final_sample = final_sampleset.samples[-1]
traj = final_sample.trajectory

{'max_overlap': [0, 0], 'ensembles': [<openpathsampling.ensemble.SingleFrameEnsemble object at 0x1111342d0>, <openpathsampling.ensemble.AndEnsemble object at 0x11079f9d0>, <openpathsampling.ensemble.SingleFrameEnsemble object at 0x11079fe10>], 'name': "I'face 0", 'greedy': False, 'min_overlap': [0, 0]}
SequentialEnsemble
__init__() got an unexpected keyword argument 'name'


UnboundLocalError: local variable 'obj' referenced before assignment

## Setting up order parameter, states, and interfaces

First we define the order parameter. We do this in two stages: first, we actually define a normal Python function which takes a `Snapshot` and return a number: in this case, the x-coordinate. Then we wrap that function into an OpenPathSampling order parameter using `paths.OP_Function`.

In [5]:
def xval(snapshot):
    """Return atom 0, coordinate 0 from the given snapshot"""
    return snapshot.xyz[0][0]
    

op_xval = paths.OP_Function(name="xval", fcn=xval)

Now we use that order parameter to define several volumes of interest. First we define the states. Note that we Python's `float("inf")` notation to create infinity (and negative infinity). The choice of where to set the state boundaries is based on the image or the PES above.

Then we set the interfaces using `LambdaVolumeSet`.

In [6]:
stateA = paths.LambdaVolume(op_xval, float("-inf"), -0.3)
stateB = paths.LambdaVolume(op_xval, 0.3, float("inf"))

interfaces = paths.VolumeFactory.LambdaVolumeSet(
    op_xval, 
    minvals=float("-inf"), 
    maxvals=[-0.30, -0.25, -0.20, -0.13, -0.05, 0.0]
)

## Equilibration in the path ensemble

At the end of the bootstrapping process, we have a set of paths which we can use to start TIS sampling. Before doing a production run, we should let these paths equilibrate within the ensembles. If you generate your initial path through dynamics that aren't the true dynamics (e.g., using high temperature runs or using metadynamics), this stage is **extremely** important. It's important here as well, but less so than if your initial path is non-physical.

First, we'll set up the `TISTransition` we're interested in. Since this particular calculation is just studying the A-to-B transition, we use this object. In general, you'll want to use one of the `TransitionNetwork` objects. See the `toy_network.ipynb` notebook for more information on studying networks of transitions.

In [7]:
tis_trans = paths.TISTransition(
    stateA=stateA,
    stateB=stateB,
    interfaces=interfaces,
    name="A->B equil",
    orderparameter=None # default order parameter for analysis; not used
)

In [8]:
initial_globalstate = paths.SampleSet.map_trajectory_to_ensembles(traj, tis_trans.ensembles)

This will be an example of a "normal" TIS run, i.e., TIS without replica exchange. We will sampling using only shooter movers (the same ones defined for the bootstrapping) and path reversal movers.

In [9]:
try:
    store_equil = paths.storage.Storage("toy_equil.nc", "w", template=template)
except RuntimeError:
    pass
equilibration = paths.PathSampling(
    storage=store_equil, 
    engine=toy_eng, 
    root_mover=tis_trans.default_movers(toy_eng),
    globalstate=initial_globalstate
)

In [10]:
equilibration.run(200)

## RETIS: Production run

Now we'll do a full production run using replica exchange TIS (RETIS).

First, we create a `RETISTransition`, in the same way that we created the `TISTransition`.

In [11]:
retis_trans = paths.RETISTransition(
    stateA=stateA,
    stateB=stateB,
    interfaces=interfaces,
    name="A->B",
    orderparameter=op_xval # we'll use this in the analysis notebook
)
start_gs = paths.SampleSet.translate_ensembles(equilibration.globalstate, retis_trans.ensembles)

At this point, we still don't have a path that satisfies the minus interface. 

In [12]:
minus_segment = equilibration.globalstate[0].trajectory
last_frame = minus_segment[-1]
extension = toy_eng.generate(last_frame, [retis_trans.minus_ensemble.can_append])
first_minus = minus_segment + extension[1:]
minus_samp = paths.Sample(
    replica=-1,
    trajectory=first_minus,
    ensemble=retis_trans.minus_ensemble,
    details=paths.MoveDetails.initialization_from_scratch(first_minus, retis_trans.minus_ensemble)
)
start_gs = start_gs.apply_samples([minus_samp])

In [13]:
import logging.config
logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

In [14]:
try:
    store_retis = paths.storage.Storage("toy_retis.nc", "w", template=template)
except RuntimeError:
    pass

store_retis.save(retis_trans) # TODO

production = paths.PathSampling(
    storage=store_retis,
    engine=toy_eng,
    root_mover=retis_trans.default_movers(toy_eng),
    globalstate=start_gs
)

In [15]:
store_retis.sync()

In [16]:
production.run(100)

In [17]:
move1 = store_retis.movepath[1].movepath.movepath.movepath.mover
move1a = store_retis.movepath[1].movepath.movepath.mover

In [18]:
print store_retis.movepath[1].contains(move1)
print store_retis.movepath[1].contains(move1a)

True
True


In [43]:
# Will only work if the particular movepath has at least one accepted sample!
# print store_retis.movepath[2].samples

[]
